In [54]:
import requests
import bs4
import csv

from pathlib import Path

In [49]:
base_url = r'https://www.funda.nl/koop/maarssen/'
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 Edg/110.0.1587.50'
}

resp = requests.get(base_url, headers=HEADERS)
html = bs4.BeautifulSoup(resp.content, 'html.parser')
print(html.prettify())
Path('tmp.html').write_text(html.prettify())

<!DOCTYPE html>
<html lang="nl">
 <head prefix="og: http://ogp.me/ns#">
  <!-- OneTrust Cookies Consent Notice start -->
  <script charset="UTF-8" data-domain-script="69c5039d-4dfc-4172-bd11-428faaabf101" data-onetrust-script="" src="https://cdn.cookielaw.org/scripttemplates/otSDKStub.js" type="text/javascript">
  </script>
  <script type="text/javascript">
   function OptanonWrapper() {
        window.dispatchEvent(new CustomEvent('cookie-consent-loaded'));
    }
  </script>
  <!-- OneTrust Cookies Consent Notice end -->
  <title>
   Koopwoningen Maarssen - Huizen te koop in Maarssen [funda]
  </title>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta charset="utf-8"/>
  <meta content="Bekijk alle 110 huizen te koop in Maarssen. Bekijk foto's, plattegronden, filter het aanbod en vind direct de huizen die je zoekt." name="description">
   <meta content="nositelinkssearchbox" name="google">
    <meta content="telephone=no" name="format-detection"/>
    <lin

527627

In [50]:
page_links = html.select('a[data-pagination-page]')
# print(page_links)
num_of_pages = max(int(a['data-pagination-page']) 
                   for a in page_links)
# num_of_pages
page_urls = [f'{base_url}p{i}' for i in range(1, num_of_pages+1)]
print(page_urls)

['https://www.funda.nl/koop/maarssen/p1', 'https://www.funda.nl/koop/maarssen/p2', 'https://www.funda.nl/koop/maarssen/p3', 'https://www.funda.nl/koop/maarssen/p4', 'https://www.funda.nl/koop/maarssen/p5', 'https://www.funda.nl/koop/maarssen/p6', 'https://www.funda.nl/koop/maarssen/p7', 'https://www.funda.nl/koop/maarssen/p8']


In [51]:
search_results = []

for i, url in enumerate(page_urls, start=1):
    print(f'Requesting page {i}...')
    resp = requests.get(url, headers=HEADERS)
    if resp.status_code == 200:
        page_html = bs4.BeautifulSoup(resp.content, 'html.parser')
        search_results.extend(page_html.find_all('li', class_='search-result'))

Requesting page 1...
Requesting page 2...
Requesting page 3...
Requesting page 4...
Requesting page 5...
Requesting page 6...
Requesting page 7...
Requesting page 8...


In [52]:
first_house = search_results[0]
first_address = first_house.find('h2', class_='search-result__header-title')
first_price = first_house.find('span', class_='search-result-price')
print(first_address.prettify())
print(first_price.prettify())
print(f'Vies adres:   |{first_address.text}|')
print(f'Schoon adres: |{first_address.text.strip()}|')
print(f'Vieze prijs:  |{first_price.text}|')
print(f'Schone prijs: |{first_price.text.strip().replace("€ ", "").replace(" k.k.", "").replace(".", "")}|')

print(first_price.text)

<h2 class="search-result__header-title fd-m-none" data-test-search-result-header-title="">
 Doornhoecklaan 2
</h2>

<span class="search-result-price">
 € 820.000 k.k.
</span>

Vies adres:   |
              Doornhoecklaan 2
        |
Schoon adres: |Doornhoecklaan 2|
Vieze prijs:  |€ 820.000 k.k.|
Schone prijs: |820000|
€ 820.000 k.k.


In [53]:
price_list = []
for house in search_results:
    address_raw = house.find('h2', class_='search-result__header-title').text
    price_raw = house.find('span', class_='search-result-price').text
    address_clean = address_raw.strip()
    price_clean = int(price_raw.replace('€ ', '') 
                               .replace(' k.k.', '') 
                               .replace(' v.o.n.', '') 
                               .replace('Price on request', '0')
                               .replace('.', '') 
                               .replace(',', ''))
    price_list.append((address_clean, price_clean))

print(f'Number of houses: {len(price_list)}')
print(price_list[:100])

Number of houses: 110
[('Doornhoecklaan 2', 820000), ('Valkenkamp 676', 575000), ('Verdilaan 11', 759000), ('Zwanenkamp 288', 580000), ('Vechtensteinlaan 5', 665000), ('Zebraspoor 582', 385000), ('Parkweg 42', 570000), ('Gaslaan 8', 695000), ('Zwanenkamp 863', 400000), ('Boomstede 665', 350000), ('Verlengde Molenweg 5', 825000), ('Fazantenkamp 850', 399000), ('Straatweg 120', 979000), ('Bloemstede 278', 398500), ('J. de Regtstraat 6', 849000), ('Fazantenkamp 417', 569000), ('Valkenkamp 252', 410000), ('Straatweg 92', 398000), ('Boomstede 100', 400000), ('Valkenkamp 751', 495000), ('Waterstede 127', 629000), ('Veenkade 16', 2250000), ('Kamelenspoor 459', 398000), ('Emmaweg 47', 695000), ('Boomstede 476', 390000), ('Binnenweg 14', 1425000), ('Straatweg 108', 549500), ('Jacob van Heemskerklaan 48', 579000), ('Zandweg 39', 1699500), ('Brederostraat 5', 495000), ('Emmaweg 1', 1749000), ('Antilopespoor 279', 250000), ('Nicolaas Maesstraat 84', 275000), ('Harmonieplein 71', 310000), ('Proostw

In [62]:
csv_file = Path('pricelist_maarssen.csv')

with open(csv_file, 'wt', encoding='utf8') as f:
    writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
    writer.writerow(['Address','Price_Euro'])
    writer.writerows(sorted(price_list))